In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim  

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS

from textblob import TextBlob

from tqdm import tqdm_notebook as tqdm
from pprint import pprint
import warnings
warnings.filterwarnings('ignore')

C:\Users\HP\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\Users\HP\Anaconda3\lib\site-packages\nltk\lm\counter.py:15: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence, defaultdict


In [2]:
df1 = pd.read_csv('ethiopia_tripadvisor_data_model_2.csv')
df1.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,22,103,126,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571


In [3]:
df2 = df1[['Destination Name', 'Review Head', 'Review Body', 'Review Text', 
           'TextBlob Polarity Head', 'TextBlob Polarity Body', 'TextBlob Polarity Text']]
df2.head()

,Destination Name,Review Head,Review Body,Review Text,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text
0,Rock-Hewn Churches of Lalibela,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,-0.05,0.300000,0.125000
1,Rock-Hewn Churches of Lalibela,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,0.25,-0.150000,-0.016667
2,Rock-Hewn Churches of Lalibela,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,0.90,0.345000,0.483750
3,Rock-Hewn Churches of Lalibela,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,0.60,0.225000,0.350000
4,Rock-Hewn Churches of Lalibela,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,0.20,0.233333,0.228571


In [4]:
from nltk import tokenize
df2['RB_Sentence1'] = df2['Review Body'].apply(lambda x: tokenize.sent_tokenize(x)[0])
df2['TextBlob Polarity RBS1'] = df2['RB_Sentence1'].map(lambda text: TextBlob(text).sentiment.polarity)
df2.head()

,Destination Name,Review Head,Review Body,Review Text,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,RB_Sentence1,TextBlob Polarity RBS1
0,Rock-Hewn Churches of Lalibela,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,-0.05,0.300000,0.125000,Amazing location must visit.,0.6000
1,Rock-Hewn Churches of Lalibela,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,0.25,-0.150000,-0.016667,I do recommend to visit Lalibela churches and ...,0.0000
2,Rock-Hewn Churches of Lalibela,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,0.90,0.345000,0.483750,A must visit in Ethiopia - one of the most uni...,0.4375
3,Rock-Hewn Churches of Lalibela,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,0.60,0.225000,0.350000,These churches have to be seen to be believed.,0.0000
4,Rock-Hewn Churches of Lalibela,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,0.20,0.233333,0.228571,Thanks to our tour organiser Ephram who is bas...,0.2000


In [5]:
df3 = pd.concat([df1, df2[['RB_Sentence1', 'TextBlob Polarity RBS1']]], axis = 1)
df3.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,RB_Sentence1,TextBlob Polarity RBS1
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000,Amazing location must visit.,0.6000
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667,I do recommend to visit Lalibela churches and ...,0.0000
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,22,103,126,0.90,0.345000,0.483750,A must visit in Ethiopia - one of the most uni...,0.4375
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000,These churches have to be seen to be believed.,0.0000
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571,Thanks to our tour organiser Ephram who is bas...,0.2000


In [6]:
df3['RB_Sentence1_Split'] = df3['RB_Sentence1'].apply(lambda x: x.split(' '))
df3.head()

,Destination Name,Destination Type,Rating,Polarity,Date,Month,Year,Year Quarter,Review Head,Review Body,Review Text,Review Head Length,Review Body Length,Review Text Length,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,RB_Sentence1,TextBlob Polarity RBS1,RB_Sentence1_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,72,179,252,-0.05,0.300000,0.125000,Amazing location must visit.,0.6000,"[Amazing, location, must, visit.]"
1,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,34,233,268,0.25,-0.150000,-0.016667,I do recommend to visit Lalibela churches and ...,0.0000,"[I, do, recommend, to, visit, Lalibela, church..."
2,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,22,103,126,0.90,0.345000,0.483750,A must visit in Ethiopia - one of the most uni...,0.4375,"[A, must, visit, in, Ethiopia, -, one, of, the..."
3,Rock-Hewn Churches of Lalibela,Landmarks,4,1,October 2019,October,2019,4,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,7,241,249,0.60,0.225000,0.350000,These churches have to be seen to be believed.,0.0000,"[These, churches, have, to, be, seen, to, be, ..."
4,Rock-Hewn Churches of Lalibela,Landmarks,5,1,October 2019,October,2019,4,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,15,243,259,0.20,0.233333,0.228571,Thanks to our tour organiser Ephram who is bas...,0.2000,"[Thanks, to, our, tour, organiser, Ephram, who..."


In [7]:
df4 = df3.copy()

In [8]:
df4 = df4.drop(['Rating'], axis  = 1)
df4 = df4.drop(['Polarity'], axis  = 1)
df4 = df4.drop(['Date'], axis  = 1)
df4 = df4.drop(['Month'], axis  = 1)
df4 = df4.drop(['Year'], axis  = 1)
df4 = df4.drop(['Year Quarter'], axis  = 1)
df4 = df4.drop(['Review Head Length'], axis  = 1)
df4 = df4.drop(['Review Body Length'], axis  = 1)
df4 = df4.drop(['Review Text Length'], axis  = 1)

In [9]:
df4 = df4[['Destination Name', 'Destination Type', 'Review Head', 'Review Body', 'Review Text', 'RB_Sentence1', 
           'TextBlob Polarity Head', 'TextBlob Polarity Body', 'TextBlob Polarity Text', 'TextBlob Polarity RBS1', 'RB_Sentence1_Split']]
df4.head()

,Destination Name,Destination Type,Review Head,Review Body,Review Text,RB_Sentence1,TextBlob Polarity Head,TextBlob Polarity Body,TextBlob Polarity Text,TextBlob Polarity RBS1,RB_Sentence1_Split
0,Rock-Hewn Churches of Lalibela,Landmarks,Incredible locations don't be put off thinking...,Amazing location must visit. This site is mark...,Incredible locations don't be put off thinking...,Amazing location must visit.,-0.05,0.300000,0.125000,0.6000,"[Amazing, location, must, visit.]"
1,Rock-Hewn Churches of Lalibela,Landmarks,It is a real miracle of the world!,I do recommend to visit Lalibela churches and ...,It is a real miracle of the world! I do recomm...,I do recommend to visit Lalibela churches and ...,0.25,-0.150000,-0.016667,0.0000,"[I, do, recommend, to, visit, Lalibela, church..."
2,Rock-Hewn Churches of Lalibela,Landmarks,Incredible experience,A must visit in Ethiopia - one of the most uni...,Incredible experience A must visit in Ethiopia...,A must visit in Ethiopia - one of the most uni...,0.90,0.345000,0.483750,0.4375,"[A, must, visit, in, Ethiopia, -, one, of, the..."
3,Rock-Hewn Churches of Lalibela,Landmarks,Amazing,These churches have to be seen to be believed....,Amazing These churches have to be seen to be b...,These churches have to be seen to be believed.,0.60,0.225000,0.350000,0.0000,"[These, churches, have, to, be, seen, to, be, ..."
4,Rock-Hewn Churches of Lalibela,Landmarks,So much history,Thanks to our tour organiser Ephram who is bas...,So much history Thanks to our tour organiser E...,Thanks to our tour organiser Ephram who is bas...,0.20,0.233333,0.228571,0.2000,"[Thanks, to, our, tour, organiser, Ephram, who..."


In [11]:
df4.to_csv('ethiopia_tripadvisor_data_model_3.csv', index = False)

### Topic Modelling

In [10]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
test_subset = df3['RB_Sentence1_Split'].values

sid = SentimentIntensityAnalyzer()
pos_word_list=[]
neu_word_list=[]
neg_word_list=[]

for value in test_subset:
    for word in value:
        print(word)
        if (sid.polarity_scores(word)['compound']) >= 0.4:
            pos_word_list.append(word)
        elif (sid.polarity_scores(word)['compound']) <= -0.4:
            neg_word_list.append(word)
        else:
            neu_word_list.append(word)                

print('Positive :', len(pos_word_list))      
print('Neutral :', len(neu_word_list))    
print('Negative :', len(neg_word_list)) 

Amazing
location
must
visit.
I
do
recommend
to
visit
Lalibela
churches
and
then
have
a
mule
trip
to
the
mountain
monastery.
A
must
visit
in
Ethiopia
-
one
of
the
most
unique
experiences
I
have
had
in
Africa.
These
churches
have
to
be
seen
to
be
believed.
Thanks
to
our
tour
organiser
Ephram
who
is
based
in
Axum
we
got
to
visit
the
churches
in
Lallibela.
Truly
a
marvel
of
early
architecture
and
craftsmanship
that
needs
to
be
experienced
in
person
to
be
fully
appreciated.
Please
I
insist
anyone
reaches
Ethiopia,,
never
miss
the
chance
to
see
Lalibela
Church's
...in
addition
to
the
church's,,
Lalibela
offers
hikking
into
beautiful
Mountains
surrounding
the
Town
Nothing
can
quite
prepare
you
for
the
rock
hewn
churches.
Go
with
a
guide
to
get
to
understand
everything.
The
rock
hewn
churches
are
just
amazing.
Easy
access
to
churches
from
Lilibela
Town
Center.
Background:
2
guys
in
our
30s
in
Addis
for
a
month,
taking
a
weekend
trip
to
Lalibela.
Lalibela
has
turned
out
as
one
of
my
favourite
d

KeyboardInterrupt: 

In [13]:
nlp = spacy.load('en_core_web_sm')

main_stop_words = ['ethiopia', 'ethiopian', 'rock', 'hewn', 'church', 'churches', 'park', 'parks', 'museum', 
                       'museums', 'omo', 'valley', 'mountain', 'mountains', 'addis', 'ababa', 'lucy', 'simien', 
                       'national', 'lalibela', 'gelada', 'haile', 'selassie', 'gondar', 'africa' , 'harar', 
                       'cathedral', 'cathedrals', 'place', 'visit', 'girmay', 
                       '\n', '‘s']
positive_stop_words = list(set(pos_word_list))
negative_stop_words = list(set(neg_word_list))
total_stop_words = main_stop_words + positive_stop_words + negative_stop_words
nlp.Defaults.stop_words.update(total_stop_words)

for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

In [36]:
# 'good' in pos_word_list
# sid.polarity_scores('good')

In [14]:
def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)

def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

nlp.add_pipe(lemmatizer,name = 'lemmatizer',after = 'ner')
nlp.add_pipe(remove_stopwords, name = 'stopwords', last = True)

In [15]:
# Adjust wanted corpus for clustering

# newest_doc = df3['Review Text']
# newest_doc = df3[df3['Polarity'] == 0]['Review Text']
# newest_doc = df3[df3['Polarity'] == 1]['Review Text']
# newest_doc = df3[df3['Rating'] == 1]['Review Text']
# newest_doc = df3[df3['Rating'] == 2]['Review Text']
# newest_doc = df3[df3['Rating'] == 3]['Review Text']
# newest_doc = df3[df3['Rating'] == 4]['Review Text']
# newest_doc = df3[df3['Rating'] == 5]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks']['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks']['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Museums']['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] < 0]['Review Text']
# newest_doc = df3[df3['TextBlob Polarity Text'] >= 0]['Review Text']
# newest_doc = df3[df3['TextBlob Polarity Text'] < 0]['Review Text']

# newest_doc = df3['Review Head']
# newest_doc = df3[df3['Polarity'] == 0]['Review Head']
# newest_doc = df3[df3['Polarity'] == 1]['Review Head']
# newest_doc = df3[df3['Rating'] == 1]['Review Head']
# newest_doc = df3[df3['Rating'] == 2]['Review Head']
# newest_doc = df3[df3['Rating'] == 3]['Review Head']
# newest_doc = df3[df3['Rating'] == 4]['Review Head']
# newest_doc = df3[df3['Rating'] == 5]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks']['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks']['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Museums']['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] >= 0]['Review Head']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] < 0]['Review Head']
# newest_doc = df3[df3['TextBlob Polarity Head'] >= 0]['Review Head']
# newest_doc = df3[df3['TextBlob Polarity Head'] < 0]['Review Head']

# newest_doc = df3['Review Body']
# newest_doc = df3[df3['Polarity'] == 0]['Review Body']
# newest_doc = df3[df3['Polarity'] == 1]['Review Body']
# newest_doc = df3[df3['Rating'] == 1]['Review Body']
# newest_doc = df3[df3['Rating'] == 2]['Review Body']
# newest_doc = df3[df3['Rating'] == 3]['Review Body']
# newest_doc = df3[df3['Rating'] == 4]['Review Body']
# newest_doc = df3[df3['Rating'] == 5]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks']['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df3['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks']['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df3['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Museums']['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] >= 0]['Review Body']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df3['TextBlob Polarity Text'] < 0]['Review Body']
# newest_doc = df3[df3['TextBlob Polarity Body'] >= 0]['Review Body']
# newest_doc = df3[df3['TextBlob Polarity Body'] < 0]['Review Body']

# newest_doc = df3['RB_Sentence1']
# newest_doc = df3[df3['Polarity'] == 0]['RB_Sentence1']
# newest_doc = df3[df3['Polarity'] == 1]['RB_Sentence1']
# newest_doc = df3[df3['Rating'] == 1]['RB_Sentence1']
# newest_doc = df3[df3['Rating'] == 2]['RB_Sentence1']
# newest_doc = df3[df3['Rating'] == 3]['RB_Sentence1']
# newest_doc = df3[df3['Rating'] == 4]['RB_Sentence1']
# newest_doc = df3[df3['Rating'] == 5]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks']['RB_Sentence1']
newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] >= 0]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Landmarks'][df2['TextBlob Polarity Text'] < 0]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks']['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] >= 0]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Nature/Parks'][df2['TextBlob Polarity Text'] < 0]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Museums']['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] >= 0]['RB_Sentence1']
# newest_doc = df3[df3['Destination Type'] == 'Museums'][df2['TextBlob Polarity Text'] < 0]['RB_Sentence1']
# newest_doc = df3[df3['TextBlob Polarity Body'] >= 0]['RB_Sentence1']
# newest_doc = df3[df3['TextBlob Polarity Body'] < 0]['RB_Sentence1']


In [16]:
doc_list = []
# Iterates through each article in the corpus.
for doc in tqdm(newest_doc):
    # Passes that article through the pipeline and adds to a new list.
    pr = nlp(doc)
    doc_list.append(pr)

# Creates, which is a mapping of word IDs to words.
words = corpora.Dictionary(doc_list)

# Turns each document into a bag of words.
corpus = [words.doc2bow(doc) for doc in doc_list]

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus,
                                           id2word = words,
                                           num_topics = 3, 
                                           random_state = 2,
                                           update_every = 1,
                                           passes = 10,
                                           alpha = 'auto',
                                           per_word_topics = True)

pprint(lda_model.print_topics(num_words = 10))

[(0,
  '0.011*"site" + 0.011*"time" + 0.010*"building" + 0.010*"George" + '
  '0.009*"hew" + 0.009*"build" + 0.009*"like" + 0.008*"world" + 0.008*"know" + '
  '0.008*"carve"'),
 (1,
  '0.031*"guide" + 0.023*"day" + 0.019*"worth" + 0.018*"tour" + 0.013*"trip" + '
  '0.008*"time" + 0.008*"monastery" + 0.007*"Orthodox" + 0.007*"walk" + '
  '0.006*"local"'),
 (2,
  '0.012*"old" + 0.012*"history" + 0.011*"experience" + 0.009*"inside" + '
  '0.008*"people" + 0.008*"lot" + 0.008*"year" + 0.007*"unique" + '
  '0.007*"religious" + 0.006*"painting"')]


In [42]:
print('\nModel Perplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model = lda_model, texts = doc_list, dictionary = words, coherence = 'c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)
# data_lemmatized = doc_list
# id2word = words


Model Perplexity:  -7.621750985229345

Coherence Score:  0.2813441978225834


In [43]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, words)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.171917  0.045843       1        1  39.238930
2     -0.037800 -0.145662       2        1  31.181807
0     -0.134117  0.099818       3        1  29.579268, topic_info=     Category        Freq         Term       Total  loglift  logprob
28    Default  230.000000        guide  230.000000  30.0000  30.0000
60    Default  171.000000          day  171.000000  29.0000  29.0000
10    Default  132.000000         tour  132.000000  28.0000  28.0000
248   Default  151.000000        worth  151.000000  27.0000  27.0000
67    Default   55.000000       George   55.000000  26.0000  26.0000
642   Default   62.000000     building   62.000000  25.0000  25.0000
35    Default  109.000000         trip  109.000000  24.0000  24.0000
26    Default   63.000000          hew   63.000000  23.0000  23.0000
68    Default   37.000000          St.   37.000000  22.0000  22.0000
218   Default   36.000000           St   36.000000  21.0000  21.0000
83    Default   54.000000        world   54.000000  20.0000  20.0000
54    Default   52.000000          lot   52.000000  19.0000  19.0000
70    Default   69.000000         like   69.000000  18.0000  18.0000
101   Default   56.000000         know   56.000000  17.0000  17.0000
433   Default   34.000000        think   34.000000  16.0000  16.0000
2     Default   49.000000       unique   49.000000  15.0000  15.0000
1     Default   96.000000   experience   96.000000  14.0000  14.0000
366   Default   33.000000      service   33.000000  13.0000  13.0000
174   Default   57.000000        carve   57.000000  12.0000  12.0000
2699  Default   40.000000       Entoto   40.000000  11.0000  11.0000
209   Default   31.000000    important   31.000000  10.0000  10.0000
193   Default   63.000000         year   63.000000   9.0000   9.0000
334   Default   41.000000        spend   41.000000   8.0000   8.0000
1560  Default   42.000000         Holy   42.000000   7.0000   7.0000
262   Default   87.000000        build   87.000000   6.0000   6.0000
82    Default   29.000000       wonder   29.000000   5.0000   5.0000
1691  Default   30.000000       Maryam   30.000000   4.0000   4.0000
21    Default   35.000000         miss   35.000000   3.0000   3.0000
155   Default   58.000000      century   58.000000   2.0000   2.0000
545   Default  113.000000          old  113.000000   1.0000   1.0000
...       ...         ...          ...         ...      ...      ...
18     Topic3    9.798200       chance   10.455823   1.1531  -6.3463
301    Topic3    9.880113  underground   10.556691   1.1519  -6.3380
1010   Topic3    7.602385          Bet    8.154284   1.1480  -6.6000
1688   Topic3    8.212113         Alem    8.813437   1.1474  -6.5229
371    Topic3   10.907760        leave   11.726449   1.1457  -6.2390
2406   Topic3    7.740591    mausoleum    8.332937   1.1444  -6.5820
1120   Topic3   10.141382         pray   10.935621   1.1427  -6.3119
1360   Topic3    9.697659   accessible   10.458362   1.1426  -6.3566
2699   Topic3   36.196915       Entoto   40.082939   1.1161  -5.0395
26     Topic3   53.028580          hew   63.329205   1.0406  -4.6577
83     Topic3   45.751484        world   54.903908   1.0357  -4.8053
2825   Topic3   16.466795      Menelik   18.899986   1.0803  -5.8272
101    Topic3   43.924923         know   56.486217   0.9666  -4.8460
174    Topic3   43.789341        carve   57.478783   0.9461  -4.8491
490    Topic3   28.383978      tourist   35.538101   0.9933  -5.2827
549    Topic3   42.320370      ancient   57.217045   0.9165  -4.8832
70     Topic3   47.616203         like   69.155334   0.8449  -4.7653
689    Topic3   21.231539       modern   26.685061   0.9895  -5.5730
155    Topic3   40.183956      century   58.805817   0.8373  -4.9350
262    Topic3   51.913574        build   87.860924   0.6919  -4.6789
340    Topic3   39.730770          way   65.952980   0.7113  -4.9464
818    Topic3   20.551125     

In [26]:
hdp_model = gensim.models.HdpModel(corpus = corpus,
                                           id2word = words,
#                                            num_topics = 3, 
                                           random_state = 2)
#                                            update_every = 1,
#                                            passes = 10,
#                                            per_word_topics = True)
pprint(hdp_model.print_topics(num_words = 10))

[(0,
  '0.002*middle + 0.002*Melka + 0.002*scafforlde + 0.002*detail + 0.002*feel + '
  '0.002*o + 0.002*equivalent + 0.002*angelic + 0.002*Setting + '
  '0.002*suffragette'),
 (1,
  '0.002*stark + 0.002*jaundiced + 0.002*Zewdu + 0.002*Mela + 0.002*shade + '
  '0.002*average + 0.002*try + 0.002*land + 0.002*Hidar + 0.002*winding'),
 (2,
  '0.002*inclusive + 0.002*upgrade + 0.002*atheist + 0.002*ton + 0.002*rise + '
  "0.002*mystique + 0.002*ge'ez + 0.001*semi + 0.001*away + 0.001*involvement"),
 (3,
  '0.002*ah + 0.002*viva + 0.002*sing + 0.002*bhirr + 0.002*warm + 0.002*Dira '
  '+ 0.002*kiss + 0.002*642 + 0.002*abebe + 0.002*business'),
 (4,
  '0.002*cut + 0.002*slit + 0.002*London + 0.002*Oblisks + 0.002*tarp + '
  '0.002*priceless + 0.002*Menlik + 0.002*reasonably + 0.002*oodle + '
  '0.002*abound'),
 (5,
  '0.002*Mek’ele + 0.002*sophistication + 0.002*Trekking + 0.002*shaped + '
  '0.002*angelic + 0.002*Mela + 0.002*Adis + 0.002*wander + 0.001*arrival + '
  '0.001*advanced'),
 (6,